In [1]:
import os; os.chdir("../"); print(os.getcwd())

/Users/accor/Desktop/scripts/projects/mlops-nba


In [3]:
import pandas as pd

In [20]:
d = pd.concat(
    [
        pd.read_parquet("data/curated/2024-01-17_2024-01-24/curated_players-20240125__164824.parquet"),
        pd.read_parquet("data/curated/first-drop/curated_players-20240125__164856.parquet"),
    ]
)


In [21]:
aggregation_functions = {
    "first": ["Age", "Pos"],
    "sum": ["G", "GS"],
}
aggreations = {
    column: key for key, value in aggregation_functions.items() for column in value
}

In [24]:
base  = d.groupby(["Player", "season", "game_type", "Tm"]).agg(
    aggreations
)
base.reset_index(inplace=True)
base

,Player,season,game_type,Tm,Age,Pos,G,GS
0,A.J. Green,2022-2023,Regular,MIL,23,SG,35,1
1,A.J. Green,2023-2024,Regular,MIL,24,SG,14,0
2,A.J. Lawson,2022-2023,Regular,DAL,22,SG,14,0
3,A.J. Lawson,2022-2023,Regular,MIN,22,SG,1,0
4,A.J. Lawson,2022-2023,Regular,TOT,22,SG,15,0
...,...,...,...,...,...,...,...,...
2867,Ziaire Williams,2023-24,Regular Season,MEM,22,F,4,0
2868,Zion Williamson,2022-2023,Regular,NOP,22,PF,29,29
2869,Zion Williamson,2023-2024,Regular,NOP,23,PF,19,19
2870,Zion Williamson,2023-24,Regular Season,NOP,23,F,3,3


In [25]:
import numpy as np
wm = lambda x: np.average(x, weights=d.loc[x.index, "G"])
means = d.groupby(["Player", "season", "game_type", "Tm"]).agg(
    {"PTS": wm, "TRB": wm, "AST": wm, "STL": wm, "BLK": wm, "TOV": wm, "PF": wm}
)
means.reset_index(inplace=True)

TypeError: Axis must be specified when shapes of a and weights differ.

In [15]:
means

,Player,season,game_type,Tm,PTS,TRB,AST,STL,BLK,TOV,PF
0,A.J. Lawson,2023-24,Regular Season,DAL,2.500000,0.500000,0.50,0.000000,0.000000,0.000000,0.000000
1,AJ Green,2023-24,Regular Season,MIL,1.500000,1.000000,0.50,0.000000,0.000000,0.000000,1.000000
2,AJ Griffin,2023-24,Regular Season,ATL,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Aaron Gordon,2023-24,Regular Season,DEN,7.333333,7.000000,2.00,0.333333,2.000000,0.666667,2.000000
4,Aaron Holiday,2023-24,Regular Season,HOU,7.500000,2.000000,2.00,1.250000,0.000000,0.500000,1.500000
...,...,...,...,...,...,...,...,...,...,...,...
436,Zach Collins,2023-24,Regular Season,SAS,8.750000,4.000000,3.25,0.500000,0.500000,2.000000,2.500000
437,Zach LaVine,2023-24,Regular Season,CHI,8.000000,2.000000,3.00,0.000000,0.000000,2.000000,4.000000
438,Zeke Nnaji,2023-24,Regular Season,DEN,2.000000,1.000000,2.00,1.000000,0.000000,2.000000,3.000000
439,Ziaire Williams,2023-24,Regular Season,MEM,4.000000,2.500000,1.50,1.250000,0.000000,1.500000,2.000000


In [17]:
means[means.Player == "LeBron James"]

,Player,season,game_type,Tm,PTS,TRB,AST,STL,BLK,TOV,PF
281,LeBron James,2023-24,Regular Season,LAL,25.666667,8.0,6.0,0.333333,0.0,3.333333,1.333333


In [18]:
d[d.Player == "LeBron James"]

,Tm,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,...,BLK,TOV,PF,PTS,season,game_type,data_period,filename,efficency,rising_stars
281,LAL,LeBron James,F,39,3,3,1955.666667,9.0,19.0,0.478333,...,0.0,3.333333,1.333333,25.666667,2023-24,Regular Season,2024-01-17_2024-01-24,20240125__164501-raw_data,26.0,False
